In [1]:
import mne
import numpy as np
import pandas as pd
from model_functions import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from Classification_models_LOSO_2Class import *

In [2]:
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold,LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

Now, we get the connectivity matrix data for :
1) P3,P4,O1,O2 nodes for delta band with n=10 epochs
2) C3,C4,F3,F4 nodes for theta band with n=10 epochs
3) C3,C4,F3,F4 nodes for delta band with n=10 epochs
4) C3,C4,T3,T4 nodes for theta band with n=10 epochs
5) C3,C4,P3,P4 nodes for theta band with n=10 epochs

In [3]:
con_mats= np.load("connectivity_4nodes_delta.npy",allow_pickle = True)
con_mats_CF_theta= np.load("theta_C3C4F3F4_connectivity.npy",allow_pickle = True)
con_mats_CF_delta= np.load("C3C3F3F4_delta_connectivity.npy",allow_pickle = True)
#I made a typo in the above npy file. It should be C3C4F3F4
con_mats_CT_theta= np.load("C3C4T3T4_theta_connectivity.npy",allow_pickle = True)
con_mats_CP_theta= np.load("C3C4P3P4_theta_connectivity.npy",allow_pickle = True)

Next, we'll form an array regarding the connectivity datas from the symmetric nodes.

In [4]:
def con_arr(i):
    PO_delta_arr = [con_mats[i][1][0], con_mats[i][2][0],con_mats[i][2][1],con_mats[i][3][0],con_mats[i][3][1],con_mats[i][3][2]] 
    CF_theta_arr = [con_mats_CF_theta[i][0][1][0], con_mats_CF_theta[i][0][2][0],con_mats_CF_theta[i][0][2][1],con_mats_CF_theta[i][0][3][0],con_mats_CF_theta[i][0][3][1],con_mats_CF_theta[i][0][3][2]] 
    CF_delta_arr = [con_mats_CF_delta[i][0][1][0], con_mats_CF_delta[i][0][2][0],con_mats_CF_delta[i][0][2][1],con_mats_CF_delta[i][0][3][0],con_mats_CF_delta[i][0][3][1],con_mats_CF_delta[i][0][3][2]] 
    CT_theta_arr = [con_mats_CT_theta[i][0][1][0], con_mats_CT_theta[i][0][2][0],con_mats_CT_theta[i][0][2][1],con_mats_CT_theta[i][0][3][0],con_mats_CT_theta[i][0][3][1],con_mats_CT_theta[i][0][3][2]] 
    CP_theta_arr = [con_mats_CP_theta[i][0][1][0], con_mats_CP_theta[i][0][2][0],con_mats_CP_theta[i][0][2][1],con_mats_CP_theta[i][0][3][0],con_mats_CP_theta[i][0][3][1],con_mats_CP_theta[i][0][3][2]]
    
    return PO_delta_arr, CF_theta_arr, CF_delta_arr, CT_theta_arr, CP_theta_arr

Next, we will form our DataFrame.

In [5]:
#For each of 36 AD and 29 CN subjects, we have mean of persistance arrays.
targets=[]
targets=[1 for i in range(36)]
targets.extend([0 for i in range(29)])

data = {"P4P3": [con_arr(i)[0][0] for i in range(0,65)],
        "O1P3": [con_arr(i)[0][1] for i in range(0,65)],
        "O1P4": [con_arr(i)[0][2] for i in range(0,65)],
        "O2P3": [con_arr(i)[0][3] for i in range(0,65)],
        "O2P4": [con_arr(i)[0][4] for i in range(0,65)],
        "O2O1": [con_arr(i)[0][5] for i in range(0,65)],
        
        "F4F3_theta": [con_arr(i)[1][0] for i in range(0,65)],
        "C3F3_theta": [con_arr(i)[1][1] for i in range(0,65)],
        "C3F4_theta": [con_arr(i)[1][2] for i in range(0,65)],
        "C4F3_theta": [con_arr(i)[1][3] for i in range(0,65)],
        "C4F4_theta": [con_arr(i)[1][4] for i in range(0,65)],
        "C4C3_theta0": [con_arr(i)[1][5] for i in range(0,65)],
        
        "F4F3_delta": [con_arr(i)[2][0] for i in range(0,65)],
        "C3F3_delta": [con_arr(i)[2][1] for i in range(0,65)],
        "C3F4_delta": [con_arr(i)[2][2] for i in range(0,65)],
        "C4F3_delta": [con_arr(i)[2][3] for i in range(0,65)],
        "C4F4_delta": [con_arr(i)[2][4] for i in range(0,65)],
        "C4C3_delta": [con_arr(i)[2][5] for i in range(0,65)],
        
        "T4T3_theta": [con_arr(i)[3][0] for i in range(0,65)],
        "C3T3_theta": [con_arr(i)[3][1] for i in range(0,65)],
        "C3T4_theta": [con_arr(i)[3][2] for i in range(0,65)],
        "C4T3_theta": [con_arr(i)[3][3] for i in range(0,65)],
        "C4T4_theta": [con_arr(i)[3][4] for i in range(0,65)],
        "C4C3_theta1": [con_arr(i)[3][5] for i in range(0,65)],
        
        "P4P3_theta": [con_arr(i)[4][0] for i in range(0,65)],
        "C3P3_theta": [con_arr(i)[4][1] for i in range(0,65)],
        "C3P4_theta": [con_arr(i)[4][2] for i in range(0,65)],
        "C4P3_theta": [con_arr(i)[4][3] for i in range(0,65)],
        "C4P4_theta": [con_arr(i)[4][4] for i in range(0,65)],
        "C4C3_theta2": [con_arr(i)[4][5] for i in range(0,65)],
        
        "Category":[targets[i] for i in range(0,65)]
       }


In [6]:
my_df= pd.DataFrame(data)
my_df

,P4P3,O1P3,O1P4,O2P3,O2P4,O2O1,F4F3_theta,C3F3_theta,C3F4_theta,C4F3_theta,...,C4T3_theta,C4T4_theta,C4C3_theta1,P4P3_theta,C3P3_theta,C3P4_theta,C4P3_theta,C4P4_theta,C4C3_theta2,Category
0,0.757260,0.810555,0.752769,0.772300,0.863025,0.864256,0.849048,0.873105,0.762283,0.789101,...,0.755040,0.854451,0.746315,0.792497,0.922489,0.806180,0.774334,0.904268,0.834879,1
1,0.605500,0.874763,0.711981,0.722660,0.877662,0.859443,0.697004,0.685780,0.469032,0.530040,...,0.458030,0.714866,0.541643,0.492991,0.730066,0.429637,0.490709,0.695885,0.653409,1
2,0.659253,0.846250,0.721886,0.742214,0.807571,0.868042,0.751026,0.689836,0.491418,0.436854,...,0.427170,0.664791,0.585614,0.404810,0.718142,0.400861,0.399869,0.739105,0.480087,1
3,0.759319,0.867188,0.814752,0.783706,0.901584,0.869455,0.632083,0.763679,0.623003,0.601804,...,0.591633,0.819341,0.596912,0.667666,0.837966,0.735637,0.678518,0.904988,0.776560,1
4,0.772771,0.887677,0.785512,0.765683,0.845178,0.878390,0.837810,0.822111,0.753706,0.736878,...,0.685136,0.816485,0.696001,0.716960,0.814071,0.678475,0.687119,0.854531,0.780376,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0.829668,0.891293,0.803339,0.804925,0.861463,0.884748,0.868784,0.887741,0.780411,0.833906,...,0.791938,0.851429,0.780961,0.846818,0.838798,0.740223,0.757558,0.818968,0.852631,0
61,0.600057,0.818826,0.686475,0.665621,0.825896,0.844612,0.738200,0.759372,0.575553,0.429355,...,0.395596,0.702193,0.598564,0.461570,0.582893,0.454023,0.379129,0.643304,0.713053,0
62,0.537257,0.846731,0.581677,0.629050,0.736418,0.756172,0.850796,0.892122,0.772279,0.679854,...,0.617055,0.710699,0.668329,0.667404,0.838365,0.612400,0.643891,0.810323,0.702097,0
63,0.404359,0.695890,0.565609,0.459311,0.748539,0.725173,0.785046,0.386158,0.294661,0.331795,...,0.293924,0.527590,0.486851,0.340366,0.558214,0.304936,0.294133,0.574622,0.369678,0


In [7]:
def train_prep(my_df, i):
    X = my_df.copy()
    y = X.Category
    
    # The following two arrays will store the target and features for the train set. We will append to it
    # as we go through various subjects.
    X_train = X.drop(index=i)
    y_train = X_train.Category
        
    # The following two arrays will store target and features for the test set. 
    X_test = pd.DataFrame(X.iloc[[i]])
    y_test = pd.DataFrame(y.iloc[[i]])    
    
    return X_train, y_train, X_test, y_test

In [8]:
def svm_cross(my_df, degree):
    confusion_matrices_train = []
    confusion_matrices_test = []
    labels = [0,1]
    
    for i in range(65):
        X_train, y_train, X_test, y_test = train_prep(my_df, i)
        
        ## Make the model object
        svc = SVC(kernel='poly', degree=degree, C=100)
        ## Fit the model object
        svc.fit(X_train[['P4P3', 'O1P3', 'O1P4', 'O2P3', 'O2P4', 'O2O1', 'F4F3_theta',
       'C3F3_theta', 'C3F4_theta', 'C4F3_theta', 'C4F4_theta', 'C4C3_theta0',
       'F4F3_delta', 'C3F3_delta', 'C3F4_delta', 'C4F3_delta', 'C4F4_delta',
       'C4C3_delta', 'T4T3_theta', 'C3T3_theta', 'C3T4_theta', 'C4T3_theta',
       'C4T4_theta', 'C4C3_theta1', 'P4P3_theta', 'C3P3_theta', 'C3P4_theta',
       'C4P3_theta', 'C4P4_theta', 'C4C3_theta2']],y_train)

        ## get the predictions
        y_train_pred = svc.predict(X_train[['P4P3', 'O1P3', 'O1P4', 'O2P3', 'O2P4', 'O2O1', 'F4F3_theta',
       'C3F3_theta', 'C3F4_theta', 'C4F3_theta', 'C4F4_theta', 'C4C3_theta0',
       'F4F3_delta', 'C3F3_delta', 'C3F4_delta', 'C4F3_delta', 'C4F4_delta',
       'C4C3_delta', 'T4T3_theta', 'C3T3_theta', 'C3T4_theta', 'C4T3_theta',
       'C4T4_theta', 'C4C3_theta1', 'P4P3_theta', 'C3P3_theta', 'C3P4_theta',
       'C4P3_theta', 'C4P4_theta', 'C4C3_theta2']].values)
        y_test_pred = svc.predict(X_test[['P4P3', 'O1P3', 'O1P4', 'O2P3', 'O2P4', 'O2O1', 'F4F3_theta',
       'C3F3_theta', 'C3F4_theta', 'C4F3_theta', 'C4F4_theta', 'C4C3_theta0',
       'F4F3_delta', 'C3F3_delta', 'C3F4_delta', 'C4F3_delta', 'C4F4_delta',
       'C4C3_delta', 'T4T3_theta', 'C3T3_theta', 'C3T4_theta', 'C4T3_theta',
       'C4T4_theta', 'C4C3_theta1', 'P4P3_theta', 'C3P3_theta', 'C3P4_theta',
       'C4P3_theta', 'C4P4_theta', 'C4C3_theta2']].values)
        
        labels=[0,1]
        confusion_matrices_train += [confusion_matrix(y_train, y_train_pred,labels=labels)]
        confusion_matrices_test += [confusion_matrix((y_test.Category).to_numpy(),y_test_pred, labels=labels)]
    
    confusion_matrices_train = np.array(confusion_matrices_train)
    confusion_matrices_test = np.array(confusion_matrices_test)
    total_confusion_train = np.sum(confusion_matrices_train, axis= 0)
    total_confusion_test = np.sum(confusion_matrices_test, axis= 0)
    
    train_metrics_dict = {'acc':accuracy(total_confusion_train), 'sens':sensitivity(total_confusion_train), 
                            'spec':specificity(total_confusion_train), 'f1':f1(total_confusion_train)}
    test_metrics_dict = {'acc':accuracy(total_confusion_test), 'sens':sensitivity(total_confusion_test), 
                            'spec':specificity(total_confusion_test), 'f1':f1(total_confusion_test)}
    return train_metrics_dict, test_metrics_dict

In [12]:
svm_cross(my_df,2)

({'acc': 1.0, 'sens': 1.0, 'spec': 1.0, 'f1': 1.0},
 {'acc': 0.8153846153846154,
  'sens': 0.8611111111111112,
  'spec': 0.7586206896551724,
  'f1': 0.8378378378378377})

In [13]:
svm_cross(my_df,3)

({'acc': 1.0, 'sens': 1.0, 'spec': 1.0, 'f1': 1.0},
 {'acc': 0.8307692307692308,
  'sens': 0.8888888888888888,
  'spec': 0.7586206896551724,
  'f1': 0.8533333333333333})